In [ ]:
import sys
sys.path.append('..')

In [ ]:
from gerumo.data.generators import build_generator, MonoGenerator
from gerumo.data.dataset import (
    load_dataset, aggregate_dataset
)
from gerumo.data.constants import (
    TELESCOPES
)
from gerumo.utils.structures import (
    Telescope, Task, ReconstructionMode
)
from gerumo.data.mappers import (
    SimpleRegression, SimpleSquareImage
)
from gerumo.config.config import get_cfg
from gerumo.models.base import build_model
%matplotlib inline

In [ ]:
dataset = load_dataset("/home/asuka/projects/gerumo2/tools/output/splits/train_events.parquet", "/home/asuka/projects/gerumo2/tools/output/splits/train_telescopes.parquet")
dataset = aggregate_dataset(dataset)

# Mono Reconstruction

In [ ]:
generator = MonoGenerator(
    dataset, Telescope("MST","MST","FlashCam"), 4,
    #SimpleSquareImage(image_channels=["image", "image_mask"], telescope_features=[], mode=ReconstructionMode.SINGLE),
    SimpleSquareImage(image_channels=["image", "image_mask"], telescope_features=["hillas_phi", "hillas_psi"], mode=ReconstructionMode.SINGLE),
    SimpleRegression(targets=["true_az", "true_alt"], domains=[(-2,2),(-2,2)]))

In [ ]:
input_shape=generator.get_input_shape()
input_shape

## Preprocessing Input

In [ ]:
%%capture
from gerumo.models.base import BaseModel
model = BaseModel(
    input_shape=input_shape, mode=ReconstructionMode.SINGLE,
    task=Task.REGRESSION, telescopes=[TELESCOPES["LST"]]
)

In [ ]:
X = model.preprocess_input(generator[0][0])
X[0].shape, X[1].shape

## Regression Model

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("config/cnn_regression.yml")
model = build_model(cfg, input_shape)

In [ ]:
model(generator[0][0])